In [128]:
# used for handle data
import pandas as pd

# used to handle geo data
from shapely.ops import transform, Point
from geopandas.tools import reverse_geocode
import geopandas as gpd

# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Optimization module in scipy
from scipy import optimize

# library written for this exercise providing additional functions for assignment submission, and others
import utils

# Plotting library
import matplotlib
matplotlib.use('nbagg')
from matplotlib import pyplot as plt


# tells matplotlib to embed plots within the notebook
# %matplotlib inline
# %matplotlib notebook


import sys
sys.path.append('./')

# import hvplot.pandas



read AIRBNB data

In [129]:
data = pd.read_csv('BigML_Dataset_5dd97775fb7bdd184c00024f.csv')
data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,calendar_last_scraped.day-of-month,calendar_last_scraped.day-of-week,first_review.year,first_review.month,first_review.day-of-month,first_review.day-of-week,last_review.year,last_review.month,last_review.day-of-month,last_review.day-of-week
0,9784567,https://www.airbnb.com/rooms/9784567,20160704013910,2016-07-04,nice Room in Amsterdam,nice Room 20 Min.to Amst.center 10 minutes wal...,NaN,nice Room 20 Min.to Amst.center 10 minutes wal...,none,nice Park metro in front bus in front,...,4,1,2016.0,1.0,2.0,6.0,2016.0,6.0,27.0,1.0
1,13163805,https://www.airbnb.com/rooms/13163805,20160704013910,2016-07-04,Best cost benefit room near a metro station,We (me and my wife) are constantly improving o...,NaN,We (me and my wife) are constantly improving o...,none,NaN,...,4,1,2016.0,6.0,13.0,1.0,2016.0,6.0,27.0,1.0
2,3719167,https://www.airbnb.com/rooms/3719167,20160704013910,2016-07-04,Nice room in A´dam SE near nature & heart centre,I offer a beautiful 19 sqm room in a house on ...,"The location is close to the shops, public tra...",I offer a beautiful 19 sqm room in a house on ...,none,A lot of different internationalities and cult...,...,4,1,2014.0,8.0,17.0,7.0,2016.0,1.0,2.0,6.0
3,13188367,https://www.airbnb.com/rooms/13188367,20160704013910,2016-07-04,Private room 15min metro ride from city center,The apartment is located a 15min ride from the...,NaN,The apartment is located a 15min ride from the...,none,NaN,...,4,1,2016.0,5.0,29.0,7.0,2016.0,5.0,31.0,2.0
4,4831606,https://www.airbnb.com/rooms/4831606,20160704013910,2016-07-04,Room in Amsterdam in a new house!! Free Bicycle,In Amsterdam Zuidoost (20 minutes to central s...,Smart tv,In Amsterdam Zuidoost (20 minutes to central s...,none,NaN,...,4,1,2015.0,1.0,2.0,5.0,2016.0,3.0,22.0,2.0


removing rows and columns that contains significant amount of nans

In [130]:
cleaned_data = data.dropna(axis='columns',thresh=0.8*data.shape[0])
cleaned_data = cleaned_data.dropna(axis='index',how='any')


In [131]:
cleaned_numeric_data = cleaned_data.select_dtypes(exclude=['object'])

In [132]:
# extracting the prices and removing them from the dt
# y = cleaned_numeric_data['price']
# cleaned_numeric_data = cleaned_numeric_data.drop(columns='price')

In [133]:
cleaned_numeric_data.plot

In [134]:
# normalize values
def featureNormalize(X):
    """
    Normalizes the features in X. returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when working with
    learning algorithms.

    Parameters
    ----------
    X : array_like
        The dataset of shape (m x n).

    Returns
    -------
    X_norm : array_like
        The normalized dataset of shape (m x n).

    """
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    n = (X.shape[0])
    mu = np.average(X, 0)
    mat1 = np.dot(mu.T[:, None], np.ones([1, n])).T
    X_norm = X_norm - mat1

    sigma = np.std(X, 0)
    mat2 = np.dot(sigma.T[:, None], np.ones([1, n])).T
    X_norm = X_norm / mat2

    return X_norm, mu, sigma

In [135]:
values = cleaned_numeric_data.values
norm_values, mean, sigma = featureNormalize(values)

normalized_data = pd.DataFrame(norm_values, index=cleaned_numeric_data.index, columns=cleaned_numeric_data.columns)
normalized_data = normalized_data.dropna(axis='columns')
normalized_data.head()

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,...,host_since.day-of-month,host_since.day-of-week,first_review.year,first_review.month,first_review.day-of-month,first_review.day-of-week,last_review.year,last_review.month,last_review.day-of-month,last_review.day-of-week
0,0.761484,0.381627,0.445549,0.643142,-0.14818,-0.14818,-2.817112,2.704464,-0.673103,-0.353720,...,-1.177259,0.108664,0.982438,-1.606073,-1.435241,0.987685,0.262603,0.263241,1.036298,-1.242037
1,1.641562,0.818840,0.445549,0.643142,-0.24134,-0.24134,-3.327567,2.717651,-0.673103,-0.353720,...,-0.608393,-0.383008,0.982438,0.030238,-0.240153,-1.147978,0.262603,0.263241,1.036298,-1.242037
2,-0.818169,-0.059765,-1.512262,-0.101024,-0.24134,-0.24134,-3.256594,2.972943,-0.673103,-0.353720,...,1.325752,1.583682,-0.855699,0.684762,0.194425,1.414818,0.262603,-3.083600,-1.744954,0.824347
4,-0.528449,0.223694,0.445549,-1.239161,-0.24134,-0.24134,-2.559634,2.390976,-1.346362,-0.353720,...,1.667072,1.583682,0.063369,-1.606073,-1.435241,0.560553,0.262603,-1.744863,0.480048,-0.828761
6,-1.632611,-0.933590,-0.337575,0.643142,-0.24134,-0.24134,-2.578703,2.862120,1.346675,1.095993,...,-0.835940,0.600337,-2.693836,1.012025,-1.109308,-0.293712,0.262603,0.263241,0.925048,1.237624


visualize the data

In [136]:
plt.figure()

ax = normalized_data.plot(x=['review_scores_rating'], y=['price'], color='b', label= 'review_scores_rating', kind='scatter')
normalized_data.plot(x=['review_scores_cleanliness'], y=['price'], color='r', label= 'review_scores_cleanliness', kind='scatter', ax=ax)
normalized_data.plot(x=['number_of_reviews'], y=['price'], color='g', label= 'number_of_reviews', kind='scatter', ax=ax)
normalized_data.plot(x=['review_scores_communication'], y=['price'], color='y', label= 'review_scores_communication', kind='scatter', ax=ax)
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

compute cost and gradient decent functions

In [137]:
def computeCost(X, y, theta):
    """
    Compute cost for linear regression. Computes the cost of using theta as the
    parameter for linear regression to fit the data points in X and y.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n+1), where m is the number of examples,
        and n is the number of features. We assume a vector of one's already
        appended to the features so we have n+1 columns.

    y : array_like
        The values of the function at each data point. This is a vector of
        shape (m, ).

    theta : array_like
        The parameters for the regression function. This is a vector of
        shape (n+1, ).

    Returns
    -------
    J : float
        The value of the regression cost function.

    Instructions
    ------------
    Compute the cost of a particular choice of theta.
    You should set J to the cost.
    """

    # initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly
    J = 0

    # ====================== YOUR CODE HERE =====================
    J = 1/(2*m)*np.dot((np.dot(X,theta) - y).T,(np.dot(X,theta) - y))

    # ===========================================================
    return J

def gradientDescent(X, y, theta, alpha, num_iters):
    """
    Performs gradient descent to learn `theta`. Updates theta by taking `num_iters`
    gradient steps with learning rate `alpha`.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n+1).

    y : arra_like
        Value at given features. A vector of shape (m, ).

    theta : array_like
        Initial values for the linear regression parameters.
        A vector of shape (n+1, ).

    alpha : float
        The learning rate.

    num_iters : int
        The number of iterations for gradient descent.

    Returns
    -------
    theta : array_like
        The learned linear regression parameters. A vector of shape (n+1, ).

    J_history : list
        A python list for the values of the cost function after each iteration.

    Instructions
    ------------
    Peform a single gradient step on the parameter vector theta.

    While debugging, it can be useful to print out the values of
    the cost function (computeCost) and gradient here.
    """
    # Initialize some useful values
    m = y.shape[0]  # number of training examples

    # make a copy of theta, to avoid changing the original array, since numpy arrays
    # are passed by reference to functions
    theta = theta.copy()

    J_history = [] # Use a python list to save cost in every iteration

    for i in range(num_iters):
        # ==================== YOUR CODE HERE =================================

        theta = theta - (alpha/m)*np.dot(X.T,(np.dot(X,theta)-y))
        # =====================================================================

        # save the cost J in every iteration
        J_history.append(computeCost(X, y, theta))

    return theta, J_history


In [138]:
# remove unrelevant features
relevant_data = cleaned_numeric_data.drop(columns=['id','scrape_id','host_id',
                                           'last_scraped.year','last_scraped.month','last_scraped.day-of-month',
                                           'last_scraped.day-of-week','host_since.month','host_since.year'
                                           ,'host_since.day-of-month','host_since.day-of-week','calendar_last_scraped.year'
                                           ,'calendar_last_scraped.month','calendar_last_scraped.day-of-month','calendar_last_scraped.day-of-week'
                                           ,'last_review.year','last_review.month','last_review.day-of-month','last_review.day-of-week'
                                           ,'first_review.year'])
                                           # ,'first_review.month','first_review.day-of-month','first_review.day-of-week'
                                           # ,'last_review.month','availability_30','availability_60','availability_90','availability_365',])

choose training data and teting data randomly

In [139]:
train_data = relevant_data.sample(frac = 0.2)

# taking out from the dataframe the price because it is the feature we try to predict
y_train = relevant_data['price'][train_data.index].values
train_data = train_data.drop(columns='price')

In [140]:
test_data = relevant_data.loc[relevant_data.index.difference(train_data.index)]
y_test = relevant_data['price'][test_data.index].values
test_data = test_data.drop(columns='price')

In [141]:
values = train_data.values
norm_values, mean, sigma = featureNormalize(values)

normalized_data = pd.DataFrame(norm_values, index=train_data.index, columns=train_data.columns)
normalized_data.head()


,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,first_review.month,first_review.day-of-month,first_review.day-of-week
7119,0.453386,0.630820,-0.222376,-0.222376,0.864259,-0.465983,-0.683333,-0.367120,-0.464164,-0.629172,...,0.717331,0.459703,0.410568,0.833973,-0.119365,-0.214757,-0.943424,0.004554,-1.444000,-1.156558
8881,0.453386,0.630820,-0.222376,-0.222376,-0.846949,0.326269,-0.683333,-0.367120,-0.464164,-0.629172,...,-1.511635,0.459703,0.410568,-0.503956,-0.119365,-0.214757,-0.708832,1.312222,0.704821,-0.729021
7229,-1.456226,0.630820,-0.222376,-0.222376,1.208435,-0.451482,0.743797,-0.367120,0.711946,0.063963,...,-0.397152,0.459703,0.410568,-0.503956,-1.329162,-0.214757,-1.017153,0.658388,-0.584472,1.408664
1273,0.453386,0.630820,-0.167309,-0.167309,0.335907,-0.312975,0.743797,1.138947,-0.464164,0.757098,...,0.717331,0.459703,0.410568,0.833973,-0.119365,-0.155742,-0.688724,0.004554,-1.229118,-1.156558
11044,0.453386,-3.716964,-0.222376,-0.222376,0.101727,-0.752455,-0.683333,-0.367120,-0.464164,-0.629172,...,-0.397152,0.459703,0.410568,0.833973,-0.119365,-0.214757,-0.661914,-1.303115,-0.477031,0.126053


In [142]:
m = train_data.shape[0]
# Add intercept term to X
X = np.concatenate([np.ones((m, 1)), normalized_data], axis=1)

building linear regression model

In [143]:
# Choose some alpha value - change this
alpha = 0.1
num_iters = 400

# init theta and run gradient descent
theta = np.zeros(X.shape[1])
theta, J_history = gradientDescent(X, y_train, theta, alpha, num_iters)

In [144]:
# Plot the convergence graph
plt.plot(np.arange(len(J_history)), J_history, lw=2)
plt.xlabel('Number of iterations')
plt.ylabel('Cost J')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Cost J')

<IPython.core.display.Javascript object>

test the model

In [145]:
values = test_data.values
norm_values, mean, sigma = featureNormalize(values)

normalized_data = pd.DataFrame(norm_values, index=test_data.index, columns=test_data.columns)
normalized_data.head()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,first_review.month,first_review.day-of-month,first_review.day-of-week
0,0.443599,0.646260,-0.156100,-0.156100,-2.836529,2.734357,-0.671131,-0.350600,-0.454137,0.041346,...,-0.403170,-0.881494,0.445868,-1.788797,-1.250378,-0.141920,-0.546359,-1.600240,-1.433189,0.989332
1,0.443599,0.646260,-0.246152,-0.246152,-3.350173,2.747638,-0.671131,-0.350600,-0.454137,-0.635517,...,0.692512,0.489587,0.445868,-0.480745,-0.077915,-0.238333,1.644843,0.036671,-0.234604,-1.145845
2,-1.526959,-0.099191,-0.246152,-0.246152,-3.278757,3.004748,-0.671131,-0.350600,-0.454137,-0.635517,...,-0.403170,0.489587,0.445868,-3.096850,-0.077915,-0.238333,-0.773035,0.691436,0.201245,1.416367
4,0.443599,-1.239292,-0.246152,-0.246152,-2.577443,2.418635,-1.335419,-0.350600,-0.454137,-0.635517,...,-0.403170,0.489587,0.445868,-1.788797,-0.077915,-0.238333,-0.656263,-1.600240,-1.433189,0.562296
6,-0.344625,0.646260,-0.246152,-0.246152,-2.596631,2.893135,1.321732,1.085985,1.850659,2.071938,...,0.692512,0.489587,0.445868,-0.480745,1.094549,-0.238333,-0.752428,1.018818,-1.106302,-0.291774


In [146]:
m = test_data.shape[0]
# Add intercept term to X
X = np.concatenate([np.ones((m, 1)), normalized_data], axis=1)

testing the model

In [147]:
predicted_price = np.dot(X, theta)

diff_from_truth = np.abs(y_test - predicted_price)
success_condition = 50 # dollar per night
success_rate = (np.sum(diff_from_truth < 50) * 100) / diff_from_truth.size
print('success rate for difference up to 50$ is: ', success_rate,'%')


success rate for difference up to 50$ is:  78.16308763784097 %


## classify cheap or expensive listing using logistic regression ML

 we chose dam square to be Amsterdam's center

In [148]:
city_center = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})
city_center['geometry'] = [Point(52.373204, 4.892694)]
city_center = city_center.to_crs({'init': 'epsg:28992'})

 cost function and optimization

In [149]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.

    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector
        or a 2-D matrix.

    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.

    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)

    # You need to return the following variables correctly
    g = np.zeros(z.shape)

    mat1 = np.ones(z.shape)
    g = mat1/(mat1 + np.exp(-z))

    return g
def costFunction(theta, X, y):
    """
    Compute cost and gradient for logistic regression.

    Parameters
    ----------
    theta : array_like
        The parameters for logistic regression. This a vector
        of shape (n+1, ).

    X : array_like
        The input dataset of shape (m x n+1) where m is the total number
        of data points and n is the number of features. We assume the
        intercept has already been added to the input.

    y : arra_like
        Labels for the input. This is a vector of shape (m, ).

    Returns
    -------
    J : float
        The computed value for the cost function.

    grad : array_like
        A vector of shape (n+1, ) which is the gradient of the cost
        function with respect to theta, at the current values of theta.

    Instructions
    ------------
    Compute the cost of a particular choice of theta. You should set J to
    the cost. Compute the partial derivatives and set grad to the partial
    derivatives of the cost w.r.t. each parameter in theta.
    """
    # Initialize some useful values
    m = y.size  # number of training examples

    J = 0
    grad = np.zeros(theta.shape)

    J = (1/m) * (np.dot((-y).T,np.log(sigmoid(np.dot(X,theta)))) - np.dot((1-y).T,np.log(1-sigmoid(np.dot(X,theta)))))
    grad = (1/m) * np.dot(X.T,(sigmoid(np.dot(X,theta))-y))

    # =============================================================
    return J, grad

 measure distance from downtown

In [150]:
# create point objects from listing's coordinates and measure distance ftom city cenetr
geometry = []
distance_from_city_center =[]
for index, row in cleaned_data.iterrows() :
    p = Point(row['latitude'], row['longitude'])
    geometry.append(p)

cleaned_data['geometry'] = geometry

cleaned_data = gpd.GeoDataFrame(cleaned_data, geometry='geometry', crs={'init': 'epsg:4326'})
cleaned_data = cleaned_data.to_crs({'init': 'epsg:28992'})

distance_from_city_center =  cleaned_data.apply(lambda row: row['geometry'].distance(city_center['geometry'][0]), axis=1)
cleaned_data['distance_from_city_center'] = distance_from_city_center
cleaned_data.head()



,id,listing_url,scrape_id,last_scraped,name,summary,description,experiences_offered,picture_url,host_id,...,first_review.year,first_review.month,first_review.day-of-month,first_review.day-of-week,last_review.year,last_review.month,last_review.day-of-month,last_review.day-of-week,geometry,distance_from_city_center
0,9784567,https://www.airbnb.com/rooms/9784567,20160704013910,2016-07-04,nice Room in Amsterdam,nice Room 20 Min.to Amst.center 10 minutes wal...,nice Room 20 Min.to Amst.center 10 minutes wal...,none,https://a2.muscache.com/im/pictures/d187b8c8-b...,27280608,...,2016.0,1.0,2.0,6.0,2016.0,6.0,27.0,1.0,POINT (6398034.500 -3664744.593),14677.424145
1,13163805,https://www.airbnb.com/rooms/13163805,20160704013910,2016-07-04,Best cost benefit room near a metro station,We (me and my wife) are constantly improving o...,We (me and my wife) are constantly improving o...,none,https://a2.muscache.com/im/pictures/07251cd1-0...,35491461,...,2016.0,6.0,13.0,1.0,2016.0,6.0,27.0,1.0,POINT (6396965.977 -3665184.067),15348.146228
2,3719167,https://www.airbnb.com/rooms/3719167,20160704013910,2016-07-04,Nice room in A´dam SE near nature & heart centre,I offer a beautiful 19 sqm room in a house on ...,I offer a beautiful 19 sqm room in a house on ...,none,https://a2.muscache.com/im/pictures/4e251c37-3...,18991270,...,2014.0,8.0,17.0,7.0,2016.0,1.0,2.0,6.0,POINT (6396568.018 -3663981.159),16321.255800
4,4831606,https://www.airbnb.com/rooms/4831606,20160704013910,2016-07-04,Room in Amsterdam in a new house!! Free Bicycle,In Amsterdam Zuidoost (20 minutes to central s...,In Amsterdam Zuidoost (20 minutes to central s...,none,https://a2.muscache.com/im/pictures/c92485ba-6...,24314617,...,2015.0,1.0,2.0,5.0,2016.0,3.0,22.0,2.0,POINT (6399225.833 -3665885.598),13054.876584
6,591951,https://www.airbnb.com/rooms/591951,20160704013910,2016-07-04,"House on waterfront, last minute reduction!",Nice house with a lot of space and fantastic v...,Nice house with a lot of space and fantastic v...,none,https://a2.muscache.com/im/pictures/fb631981-9...,2580855,...,2012.0,9.0,5.0,3.0,2016.0,6.0,26.0,7.0,POINT (6398185.274 -3663811.834),15097.979942


In [151]:
print(cleaned_data['room_type'])

0           Private room
1           Private room
2           Private room
4           Private room
6        Entire home/apt
              ...       
13828    Entire home/apt
13829       Private room
13830       Private room
13831       Private room
13832    Entire home/apt
Name: room_type, Length: 8615, dtype: object


In [152]:
# we want room type to be 1 for 'home/apt' else 0
room_type = cleaned_data['room_type'] == 'Entire home/apt'

X = np.vstack([cleaned_data['distance_from_city_center'].values,room_type]).T

print(room_type.size-np.sum(room_type))

1697


 create cheap\expensive feature = y

In [153]:
expensive_condition = 100
expensive = cleaned_data['price'] > 100
cleaned_data['expensive'] = expensive

y = expensive



 visualize the data

In [154]:
def plotData(X, y):
    """
    Plots the data points X and y into a new figure. Plots the data
    points with * for the positive examples and o for the negative examples.

    Parameters
    ----------
    X : array_like
        An Mx2 matrix representing the dataset.

    y : array_like
        Label values for the dataset. A vector of size (M, ).

    Instructions
    ------------
    Plot the positive and negative examples on a 2D plot, using the
    option 'k*' for the positive examples and 'ko' for the negative examples.
    """
    # Create New Figure
    fig = plt.figure()

    # ====================== YOUR CODE HERE ======================
    # Find Indices of Positive and Negative Examples
    pos = y == 1
    neg = y == 0

    # Plot Examples
    plt.plot(X[pos, 0], X[pos, 1], 'k*', lw=2, ms=10)
    plt.plot(X[neg, 0], X[neg, 1], 'ko', mfc='y', ms=8, mec='k', mew=1)

In [156]:
plotData(X, y)
# add axes labels
plt.xlabel('distance from city center')
plt.ylabel('room type')
plt.legend(['expensive', 'cheap'])



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 choosing train and test data

 normalize the data

 concatenate X

using the model

 predict and success rate